In [3]:
import astropy
from astropy.io import fits 
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time

In [4]:
#Goal: define 2 functions to apply to TESS data
#     1. dataClean
#      -remove nan values 
#      -remove flagged values 
#      -correct time values

#     2. flareFinder
#       -compare avg flux value of light curve to flux data
#       -if value_flux > avg flux by x(sigma), pull into new array (until equal to or less than x(sigma)) (new array for each flare?)
#       -then continue through the rest of the data
#       -find max flux in each array 
#       -pull associated timestamp from cleaned data 
#       -print timestamp, flux value for each flare discovered

In [42]:
def dataClean(filename): #Can I use the file path as an input?
    """ This function will remove nan values from TESS 
        20 second lightcurve data and correct time values"""
    
 #Create an array of time and flux data with nans removed    
    with fits.open(filename, mode="readonly") as hdulist:
        raw_time = hdulist[1].data['TIME']
        raw_flux = hdulist[1].data['PDCSAP_FLUX']
    
    data = np.vstack((raw_time, raw_flux))
    nonan_data = data[:, ~np.isnan(data).any(axis=0)]

#Apply time correction and create array of corrected and cleaned data
    times = nonan_data[0]
    flux = nonan_data[1]

    t_corr = []
    for i in times:
        r = i + 2457000
        t_corr.append(r)
    time = Time(t_corr, format = 'jd', scale = 'utc')
    time.format = 'iso'
    time = np.array(time)
    flux = np.array(flux)
    
#Return cleaned data
    return[time,flux]

In [159]:
def flareFinder(cleaned_data, sigma_multiplier):
    """ This function will take cleaned data and a multiple to be applied to sigma. 
        The function will run through the data, creating arrays of flux values 
        when the flux is > avg flux by the specified multiple of sigma. Using the max
        flux value from each array, the corresponding time value will be pulled from
        the cleaned data set. Timestamp and max flux will be printed """

#find avg flux and sigma from tess data, set as variables, using random number for now
    flux = cleaned_data[1]
    time = cleaned_data[0]
    
#NEXT: ix = np.where(flux > avg_flux + sigma_multiplier*sigma)
    ix = np.where((flux > 4180)&(flux < 4800))
    flareFlux = flux[ix]
    flareTime = time[ix]
    
    return[flareFlux,flareTime]

In [160]:
TESSDATA = '/Users/katborski/Documents/GitHub/AFPSC/TESS/tess2021232031932-s0042-0000000250081915-0213-a_fast-lc.fits'
cleaned_data = dataClean(TESSDATA)

In [167]:
flares = flareFinder(cleaned_data)

In [168]:
print(len(flares[0]))
print(len(flares[1]))

15222
15222
